In [1]:
#importing the needed libraries
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_validate, KFold
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, make_scorer, recall_score, f1_score, confusion_matrix, accuracy_score

In [2]:
#reading in the data we will use in the machine learning pipeline, with the top ten features of the method
df = pd.read_excel('pro_impedance_training_set_10_final.xlsx')
df

,Pro_exp_dur_sd,Pro_exp_diff_mean,Pro_RRV_RMSSD,Pro_ie_ratio_sd,Pro_duty_cycle_mean,Pro_exp_dur_mean,Pro_ie_ratio_mean,Pro_insp_flow_sd,Pro_insp_dur_sd,Pro_RSP_Rate_Mean,Task_Label,Participant,Classification
0,0.335261,0.319571,1824.206677,0.410715,0.450117,2.005375,0.885292,0.366389,0.981967,16.085184,12.0a,10785,0
1,0.161704,0.182625,391.484902,0.212815,0.462781,1.830889,0.883762,0.194970,0.309334,17.667177,12.0b,10785,0
2,0.521450,0.375667,325.361645,0.112015,0.392339,2.599857,0.652974,0.295719,0.175999,14.340384,12.0c,10785,0
3,0.251297,0.321000,313.146611,0.180558,0.444252,2.095857,0.819468,0.155977,0.281658,15.633870,12.0d,10785,0
4,1.591572,1.866800,2873.485775,0.222939,0.389224,3.073333,0.669187,0.062505,0.141959,10.764507,12.0e,10785,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2497,0.147065,0.122000,584.235642,0.376923,0.469096,1.999375,0.934994,3.228809,0.551583,16.020141,85,94527,0
2498,1.226542,1.490429,1986.574774,1.054285,0.520104,1.910250,1.471548,8.055498,0.656451,17.989214,87,94527,1
2499,0.269661,0.229333,713.152041,0.293774,0.428940,2.268000,0.792047,6.827633,0.543736,15.266439,89,94527,0
2500,1.135101,0.757286,978.508644,0.269589,0.346931,2.470125,0.577715,16.213339,0.330195,16.641077,91,94527,1


In [3]:
# extracting the features from the DataFrame
features = df.drop(columns=['Classification', 'Participant', 'Task_Label'])

# extracting the Classification (0/1) and participants from the dataframe
target = df['Classification'].values
participants = df['Participant'].values
task = df['Task_Label'].values

# extracting the feature names
feature_names = features.columns.tolist()

# converting the features, target, participants to numpy arrays
features_array = features.values
target_array = target

# creating the dictionary with the needed format to enter into the nested cross-validation
data_dict = {
    'data': features_array,
    'target': target_array,
    'feature_names': feature_names,
    'participants': participants,
    'task': task
}

data_dict

{'data': array([[3.35260700e-01, 3.19571400e-01, 1.82420668e+03, ...,
         3.66389500e-01, 9.81967200e-01, 1.60851837e+01],
        [1.61704400e-01, 1.82625000e-01, 3.91484902e+02, ...,
         1.94969900e-01, 3.09333900e-01, 1.76671773e+01],
        [5.21449700e-01, 3.75666700e-01, 3.25361645e+02, ...,
         2.95718700e-01, 1.75999400e-01, 1.43403844e+01],
        ...,
        [2.69660600e-01, 2.29333300e-01, 7.13152041e+02, ...,
         6.82763330e+00, 5.43735900e-01, 1.52664393e+01],
        [1.13510080e+00, 7.57285700e-01, 9.78508644e+02, ...,
         1.62133392e+01, 3.30194600e-01, 1.66410766e+01],
        [4.31400800e-01, 3.58571400e-01, 4.89504122e+02, ...,
         6.05089280e+00, 1.84028400e-01, 1.79313024e+01]]),
 'target': array([0, 0, 0, ..., 0, 1, 0]),
 'feature_names': ['Pro_exp_dur_sd',
  'Pro_exp_diff_mean',
  'Pro_RRV_RMSSD',
  'Pro_ie_ratio_sd',
  'Pro_duty_cycle_mean',
  'Pro_exp_dur_mean',
  'Pro_ie_ratio_mean',
  'Pro_insp_flow_sd',
  'Pro_insp_dur_sd',
 

In [4]:
#running the nested cross-validation for the Accelerometer method using XGBoost
from sklearn.metrics import accuracy_score, precision_score, recall_score
import xgboost as xgb

# random seed for reproducibility
random_seed = 1
np.random.seed(random_seed)

X = data_dict['data']
Y = data_dict['target']
groups = data_dict['participants']

# setting up the parameter grid: the combinations of these will be tried out for ever outer loop's inner loop
param_grid = {
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'subsample': [0.9, 1.0],
    'max_depth': [3, 4],
    'min_child_weight': [1, 2],
    'gamma': [0, 1],
    'colsample_bytree': [0.8, 1.0],
}


xgbc = xgb.XGBClassifier(random_state=random_seed)

logo = LeaveOneGroupOut()


# defining the techniques to use for the inner and outer loops
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1)
outer_cv = logo.split(X, Y, groups)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='macro'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, average='weighted')
}

# performing nested cross-validation
clf = GridSearchCV(estimator=xgbc, param_grid=param_grid, cv=inner_cv)
cv_dic = cross_validate(clf, X, Y, cv=outer_cv, scoring=scoring, return_estimator=True, return_train_score=False)
mean_acc_score = cv_dic['test_accuracy'].mean()
mean_prec_score = cv_dic['test_precision'].mean()
mean_rec_score = cv_dic['test_recall'].mean()
mean_f1_score = cv_dic['test_f1_score'].mean()
mean_roc_auc = cv_dic['test_roc_auc'].mean()  # Mean ROC AUC score


print('nested_accuracy_scores:   ', cv_dic['test_accuracy'])
print('mean score:            {0:.4f}'.format(mean_acc_score))

print('nested_precision_scores:   ', cv_dic['test_precision'])
print('mean score:            {0:.4f}'.format(mean_prec_score))

print('nested_recall_scores:   ', cv_dic['test_recall'])
print('mean score:            {0:.4f}'.format(mean_rec_score))

print('nested_f1_scores:   ', cv_dic['test_f1_score'])
print('mean score:            {0:.4f}'.format(mean_f1_score))

print('nested_roc_auc_scores:   ', cv_dic['test_roc_auc'])
print('mean score:            {0:.4f}'.format(mean_roc_auc))

nested_accuracy_scores:    [0.86363636 0.86363636 0.89393939 0.90909091 0.7        0.92424242
 0.72727273 0.95454545 0.86363636 0.78787879 0.78787879 0.86363636
 0.92424242 0.90909091 0.96969697 0.77272727 0.87878788 0.92424242
 0.87878788 0.93939394 0.98484848 0.81818182 0.90909091 0.90909091
 0.86363636 0.75757576 0.62121212 0.53030303 0.95454545 0.68181818
 0.90909091 0.93939394 0.84848485 0.96969697 0.90909091 0.68181818
 0.59090909 0.83333333]
mean score:            0.8434
nested_precision_scores:    [0.86694493 0.87471352 0.9000259  0.92045455 0.75384615 0.92981093
 0.74848485 0.95538803 0.86518847 0.79265734 0.78787879 0.86518847
 0.92398198 0.91223776 0.97107438 0.7689097  0.87809917 0.92611833
 0.88234266 0.94466403 0.98545455 0.87878788 0.90909091 0.90909091
 0.90082645 0.75636364 0.63196063 0.6225756  0.95538803 0.67445887
 0.92727273 0.94008264 0.8486166  0.97107438 0.90909091 0.71900826
 0.6021756  0.88571429]
mean score:            0.8551
nested_recall_scores:    [0.86363

In [5]:
sensitivity_scores = []
specificity_scores = []

# iteration over the outer folds to calculate average sensitivity and specificity
for i, (train_index, test_index) in enumerate(logo.split(X, Y, groups)):
    print(f"Fold {i + 1}:")

    # getting the trained estimator for the present fold
    estimator = cv_dic['estimator'][i]
    
    # predicting on the test set
    y_pred = estimator.predict(X[test_index])
    
    # creating confusion matrix
    tn, fp, fn, tp = confusion_matrix(Y[test_index], y_pred).ravel()
    
    # calculating the sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print("=" * 30)
    
    # store the sensitivity and specificity scores
    sensitivity_scores.append(sensitivity)
    specificity_scores.append(specificity)

    
    # calculate and print the averages of the sensitivity and specificity scores previously calculated
average_sensitivity = np.mean(sensitivity_scores)
average_specificity = np.mean(specificity_scores)

print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)

Fold 1:
Sensitivity: 0.7083
Specificity: 0.9524
Fold 2:
Sensitivity: 0.6667
Specificity: 0.9762
Fold 3:
Sensitivity: 0.9167
Specificity: 0.8810
Fold 4:
Sensitivity: 0.7500
Specificity: 1.0000
Fold 5:
Sensitivity: 0.2917
Specificity: 0.9722
Fold 6:
Sensitivity: 0.9583
Specificity: 0.9048
Fold 7:
Sensitivity: 0.7500
Specificity: 0.7143
Fold 8:
Sensitivity: 0.9583
Specificity: 0.9524
Fold 9:
Sensitivity: 0.8333
Specificity: 0.8810
Fold 10:
Sensitivity: 0.7500
Specificity: 0.8095
Fold 11:
Sensitivity: 0.7083
Specificity: 0.8333
Fold 12:
Sensitivity: 0.8333
Specificity: 0.8810
Fold 13:
Sensitivity: 0.8750
Specificity: 0.9524
Fold 14:
Sensitivity: 0.9167
Specificity: 0.9048
Fold 15:
Sensitivity: 0.9167
Specificity: 1.0000
Fold 16:
Sensitivity: 0.5833
Specificity: 0.8810
Fold 17:
Sensitivity: 0.7917
Specificity: 0.9286
Fold 18:
Sensitivity: 0.8333
Specificity: 0.9762
Fold 19:
Sensitivity: 0.8750
Specificity: 0.8810
Fold 20:
Sensitivity: 0.8333
Specificity: 1.0000
Fold 21:
Sensitivity: 1.0000


In [6]:
#print all performance scores that might be of interest
import scipy.stats

def calculate_stats(data):
    mean = np.mean(data)
    sd = np.std(data, ddof=1)  # using sample standard deviation (N-1)
    ci = scipy.stats.norm.interval(0.95, loc=mean, scale=sd/np.sqrt(len(data)))
    min_val = np.min(data)
    max_val = np.max(data)
    return mean, sd, ci, min_val, max_val

keys = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1_score', 'test_roc_auc', 'sensitivity_scores', 'specificity_scores']

for key in keys:
    if key in cv_dic:
        data = cv_dic[key]
    elif key == 'sensitivity_scores':
        data = sensitivity_scores
    elif key == 'specificity_scores':
        data = specificity_scores

    stats = calculate_stats(data)
    print(f"{key}: Mean={stats[0]}, SD={stats[1]}, 95% CI={stats[2]}, Min={stats[3]}, Max={stats[4]}")


test_accuracy: Mean=0.8433811802232855, SD=0.1120273314006229, 95% CI=(0.8077622982462472, 0.8790000622003237), Min=0.5303030303030303, Max=0.9848484848484849
test_precision: Mean=0.855132705658304, SD=0.10266147726436874, 95% CI=(0.8224916796109084, 0.8877737317056996), Min=0.6021756021756022, Max=0.9854545454545454
test_recall: Mean=0.8433811802232855, SD=0.1120273314006229, 95% CI=(0.8077622982462472, 0.8790000622003237), Min=0.5303030303030303, Max=0.9848484848484849
test_f1_score: Mean=0.8305651383307888, SD=0.11959368537556249, 95% CI=(0.792540548158613, 0.8685897285029647), Min=0.5301951779563719, Max=0.9837718219818048
test_roc_auc: Mean=0.9164317042606516, SD=0.10139778076344416, 95% CI=(0.8841924681689612, 0.9486709403523419), Min=0.6051587301587301, Max=1.0
sensitivity_scores: Mean=0.7993421052631579, SD=0.1696952486030504, 95% CI=(0.7453878161045516, 0.8532963944217641), Min=0.2916666666666667, Max=1.0
specificity_scores: Mean=0.8695697577276524, SD=0.12923048959803374, 95%

In [7]:
# IF this is the best-performing nested CV for the method: print best hyperparameters for each outer loop
for fold_idx, estimator in enumerate(cv_dic['estimator']):
    print(f'Fold {fold_idx + 1} - Best hyperparameters: {estimator.best_params_}')

Fold 1 - Best hyperparameters: {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 1.0}
Fold 2 - Best hyperparameters: {'colsample_bytree': 1.0, 'gamma': 1, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 200, 'subsample': 0.9}
Fold 3 - Best hyperparameters: {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 0.9}
Fold 4 - Best hyperparameters: {'colsample_bytree': 1.0, 'gamma': 1, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 200, 'subsample': 0.9}
Fold 5 - Best hyperparameters: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 100, 'subsample': 1.0}
Fold 6 - Best hyperparameters: {'colsample_bytree': 0.8, 'gamma': 1, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200, 'sub

In [8]:
# IF this is the best-performing nested CV for the method: print mode of hyperparameters

from collections import defaultdict, Counter

# initialize a dictionary to contain all hyperparameters and their values across loops
hyperparameters = defaultdict(list)

# extracting hyperparameters of each outer loop
for estimator in cv_dic['estimator']:
    for param, value in estimator.best_params_.items():
        hyperparameters[param].append(value)

# count the most frequent value for each hyperparameter
for param, values in hyperparameters.items():
    most_common_value, count = Counter(values).most_common(1)[0]  # Get the most frequent value
    print(f'Hyperparameter: {param} - Most frequent value: {most_common_value} (Count: {count})')

Hyperparameter: colsample_bytree - Most frequent value: 1.0 (Count: 21)
Hyperparameter: gamma - Most frequent value: 1 (Count: 21)
Hyperparameter: learning_rate - Most frequent value: 0.1 (Count: 38)
Hyperparameter: max_depth - Most frequent value: 3 (Count: 22)
Hyperparameter: min_child_weight - Most frequent value: 2 (Count: 21)
Hyperparameter: n_estimators - Most frequent value: 100 (Count: 31)
Hyperparameter: subsample - Most frequent value: 0.9 (Count: 31)
